# Analysis

## Prepare Data

### Import Libraries

In [ ]:
import os
import numpy as np
from scipy import io, stats
import mne

### Import Data

In [ ]:
# NOTE import 1 subject
subject = 'NI'
id = '72_9'
save_path = "D:\Files\Master Degree\Master Thesis\Code\Simulation_Result\\72 NI_10\\9"

# import all subjects
# subject = 'NI_AR'
# id = '73'
# o_folder = 'D:\Files\Master Degree\Master Thesis\Code\Simulation_Result\\70 AR_10'
# y_folder = 'D:\Files\Master Degree\Master Thesis\Code\Simulation_Result\\72 NI_10'
# save_path = "D:\Files\Master Degree\Master Thesis\Code\Simulation_Result\\73 NI_AR_3"
# filestr = 'EEGpower20sEpochs' # 'TimeStatwithin20sResult' / EEGpowerRaw_allNodes

# Nodes in PFC
chosen_PFC = [2, 10, 12, 16, 17, 18, 25, 26, 30, 36, 44, 46, 50, 51, 52, 59, 60, 64]

### Helper Function

In [ ]:
def divide_data (subs, subs_time, total, cutoff=0, stim_start=0, stim_end=0):
  
  start_point = np.where(subs_time == cutoff)[0][0] + 1
  if stim_start != 0:
    stim_period = np.where(subs_time == stim_start)[0][0] + 1 # (stim_start / 3.90625) - 1
    stim_post = np.where(subs_time == stim_end)[0][0] + 1 # (stim_end / 3.90625) - 1
    subs_all = subs[start_point:,:]
    subs_pre = subs[start_point:stim_period,:]
    subs_stim = subs[stim_period:stim_post, :]
    if total == stim_end:
      all_subs = [subs_all, subs_pre, subs_stim]
      return all_subs
    subs_post = subs[stim_post:, :]
    all_subs = [subs_all, subs_pre, subs_stim, subs_post]
  else: 
    subs = subs[start_point:,:]
    all_subs = [subs]

  return all_subs

def computed_psd_raw(raw, fmax=np.inf):
  spectrum = raw.compute_psd(method='welch', fmax=fmax , picks='all', n_fft=256*4, n_per_seg=256*4)
  psds, freqs = spectrum.get_data(return_freqs=True, picks='all')

  return spectrum, [psds, freqs]

def computed_psd_eachEpochs(epochs, fmax=np.inf):
  epo_spectrum = epochs.compute_psd(method='welch', fmax=fmax ,picks='all', n_fft=256*4, n_per_seg=256*4)
  psds, freqs = epo_spectrum.get_data(return_freqs=True, picks='all')

  return epo_spectrum, [psds, freqs]

def kstest(data):
  result = stats.ks_1samp(data, stats.norm.cdf)

  return result

def bandpower(psd, f, band):
    """
    https://raphaelvallat.com/bandpower.html
    Compute the average power of the signal x in a specific frequency band.

    Parameters
    ----------
    data : 1d-array
        Input signal in the time-domain.
    sf : float
        Sampling frequency of the data.
    band : list
        Lower and upper frequencies of the band of interest.

    Return
    ------
    bp : float
        Absolute or relative band power.
    """
    from scipy.integrate import simps
    band = np.asarray(band)
    low, high = band

    # Frequency resolution
    freq_res = f[1] - f[0]

    # Find closest indices of band in frequency vector
    idx_band = np.logical_and(f >= low, f <= high)

    # Integral approximation of the spectrum using Simpson's rule.
    absolutep = simps(psd[idx_band], dx=freq_res)
    total_power = simps(psd, dx=freq_res)
    relative = absolutep / total_power
    
    return absolutep, relative

# calculate EEG power for each node in each segment
def powerEEGraw(data, prefix, mode='99'):

  eegpow = {}

  for p in range(len(prefix)):
    dat = data[prefix[p]]
    power = []
    for i in range(68):
      absolute_swa, relative_swa = bandpower(psd=dat[0][0][i], f=dat[0][1][0], band=[0.5, 4])
      absolute_os, relative_os = bandpower(psd=dat[0][0][i], f=dat[0][1][0], band=[0.5, 1])
      absolute_delta, relative_delta = bandpower(psd=dat[0][0][i], f=dat[0][1][0], band=[1, 4])
      absolute_theta, relative_theta = bandpower(psd=dat[0][0][i], f=dat[0][1][0], band=[4, 8])
      absolute_slowspin, relative_slowspin = bandpower(psd=dat[0][0][i], f=dat[0][1][0], band=[8, 12])
      absolute_fastspin, relative_fastspin = bandpower(psd=dat[0][0][i], f=dat[0][1][0], band=[12, 15])
      absolute_rest, relative_rest = bandpower(psd=dat[0][0][i], f=dat[0][1][0], band=[15, 130])

      abpow = [absolute_swa, absolute_os, absolute_delta, absolute_theta, absolute_slowspin, absolute_fastspin,absolute_rest]
      relpow = [relative_swa, relative_os, relative_delta, relative_theta, relative_slowspin, relative_fastspin, relative_rest]
      power.append([abpow, relpow])

    eegpow[prefix[p]] = np.array(power, dtype=object)

  eegpower = save_path + '/EEGpowerRaw_' + mode + '_' + subject + '_' + id + '.mat'
  io.savemat(eegpower, mdict=eegpow)

  return eegpow, eegpower

def mean_power(data, prefix, axis=0):
  """
  result: (simulation round, power, frequency band)
  power: 0 = absolute power, 1 = relative power
  frequency band: 0 = swa, 1 = os, 2 = delta, 3 = theta, 4 = slowspin, 5 = fastspin, 6 = rest
  """
  new = {}
  for i in prefix:
    new[i] = data[i].mean(axis=axis)

  return new

def getallEEGpow(path, filestr):

  directory = os.fsencode(path)
  subjects_psd = {}
  all = []
  pre = []
  stim = []
  post = []

  for i, folder in enumerate(os.listdir(directory)):
    foldername = os.fsdecode(folder)
    if foldername == 'desktop.ini' or 'within' in foldername:
      continue
    subpath = path + '/' + foldername
    subdirec = os.fsencode(subpath)

    for j, file in enumerate(os.listdir(subdirec)):
      filename = os.fsdecode(file)
      filename_path = subpath + '/' + filename
      if filename.startswith(filestr) and filename.endswith(".mat"):
        psdsub = io.loadmat(filename_path)
        # all.append(psdsub['psd raw all'])
        # pre.append(psdsub['psd raw prestim'])
        # stim.append(psdsub['psd raw stimon'])
        # post.append(psdsub['psd raw poststim'])
        all.append(psdsub['psd epo all'])
        pre.append(psdsub['psd epo prestim'])
        stim.append(psdsub['psd epo stimon'])
        post.append(psdsub['psd epo poststim'])

  subjects_psd['power all'] = np.array(all, dtype=object)
  subjects_psd['power pre'] = np.array(pre, dtype=object)
  subjects_psd['power stim'] = np.array(stim, dtype=object)
  subjects_psd['power post'] = np.array(post, dtype=object)

  return subjects_psd

# compute mean of relative power each rounds PFC, 3 bands
def meanstdpfc_power(data, prefix, chosen=chosen_PFC):
  new = {}
  for i in prefix:
    m = data[i][:, chosen, 1, :3].mean(axis=1)
    s = np.std(data[i][:, chosen, 1, :3].astype(np.float64), axis=1)

    new[i] = np.array([m, s])

  return new

# calculate EEG power for each node in each epoch 
def powerEEGepo(data, prefix):

  eegpow = {}

  for p in range(len(prefix)):
    dat = data[prefix[p]]
    num = dat[0][0].shape[0]
    allpow = []
    for i in range(num):
      power = []
      for j in range(68):
        absolute_swa, relative_swa = bandpower(psd=dat[0][0][i][j], f=dat[0][1][0], band=[0.5, 4])
        absolute_os, relative_os = bandpower(psd=dat[0][0][i][j], f=dat[0][1][0], band=[0.5, 1])
        absolute_delta, relative_delta = bandpower(psd=dat[0][0][i][j], f=dat[0][1][0], band=[1, 4])
        absolute_theta, relative_theta = bandpower(psd=dat[0][0][i][j], f=dat[0][1][0], band=[4, 8])
        absolute_slowspin, relative_slowspin = bandpower(psd=dat[0][0][i][j], f=dat[0][1][0], band=[8, 12])
        absolute_fastspin, relative_fastspin = bandpower(psd=dat[0][0][i][j], f=dat[0][1][0], band=[12, 15])
        absolute_rest, relative_rest = bandpower(psd=dat[0][0][i][j], f=dat[0][1][0], band=[15, 130])

        abpow = [absolute_swa, absolute_os, absolute_delta, absolute_theta, absolute_slowspin, absolute_fastspin,absolute_rest]
        relpow = [relative_swa, relative_os, relative_delta, relative_theta, relative_slowspin, relative_fastspin, relative_rest]
        power.append([abpow, relpow])

      allpow.append(power)
    eegpow[prefix[p]] = np.array(allpow, dtype=object)

  eegpower = save_path + '/EEGpower12sEpochs_' + subject + '_' + id + '.mat'
  io.savemat(eegpower, mdict=eegpow)

  return eegpow, eegpower

def convert_zscore(data, prefix, axis=None):
  # concatenate pre-stim-post together
  con_data = np.concatenate((data[prefix[1]], data[prefix[2]], data[prefix[3]]), axis=1)
  znew = stats.zscore(con_data.astype(np.float64), axis=axis)

  return znew

def prep_FC(FC):

  fc = []
  for i in range(10):
    FC[i][np.diag_indices_from(FC[i])] = 0 # changing the diagonal (self_connections) to 0.
    idxs = np.triu_indices_from(FC[i], k=1)
    # choosing only upper traingle of FC matrix 
    triu_FC = np.asarray(FC[i][idxs])
    fc.append(triu_FC)

  return fc

def getallfc(path, filestr):

  directory = os.fsencode(path)
  subjects_psd = {}
  all = []
  pre = []
  stim = []
  post = []
  pfcall = []
  pfcpre = []
  pfcstim = []
  pfcpost = []

  for i, folder in enumerate(os.listdir(directory)):
    foldername = os.fsdecode(folder)
    if foldername == 'desktop.ini' or 'within' in foldername:
      continue
    subpath = path + '/' + foldername
    subdirec = os.fsencode(subpath)

    for j, file in enumerate(os.listdir(subdirec)):
      filename = os.fsdecode(file)
      filename_path = subpath + '/' + filename
      if filename.startswith(filestr) and filename.endswith(".mat"):
        psdsub = io.loadmat(filename_path)
        all.append(psdsub['all fc'])
        pre.append(psdsub['pre fc'])
        stim.append(psdsub['stim fc'])
        post.append(psdsub['post fc'])
        pfcall.append(psdsub['pfc fc'])
        pfcpre.append(psdsub['pfc pre fc'])
        pfcstim.append(psdsub['pfc stim fc'])
        pfcpost.append(psdsub['pfc post fc'])

  subjects_psd['all fc'] = np.array(all, dtype=object)
  subjects_psd['pre fc'] = np.array(pre, dtype=object)
  subjects_psd['stim fc'] = np.array(stim, dtype=object)
  subjects_psd['post fc'] = np.array(post, dtype=object)
  subjects_psd['pfc fc'] = np.array(pfcall, dtype=object)
  subjects_psd['pfc pre fc'] = np.array(pfcpre, dtype=object)
  subjects_psd['pfc stim fc'] = np.array(pfcstim, dtype=object)
  subjects_psd['pfc post fc'] = np.array(pfcpost, dtype=object)

  return subjects_psd

def ind_ttest(data1, data2, alternative='two-sided', axis=0):
  norm1 = stats.shapiro(data1.flatten())
  norm2 = stats.shapiro(data2.flatten())

  homo = stats.levene(data1.flatten(), data2.flatten())

  n1 = data1.flatten().shape[0]
  n2 = data1.flatten().shape[0]
  df = (n1 + n2) - 2 

  if homo[1] >= 0.05:
    test = stats.ttest_ind(a=data1, b=data2, axis=axis, equal_var=True, alternative=alternative) # default is 2 tailed
  else:
    test = stats.ttest_ind(a=data1, b=data2, axis=axis, equal_var=False, alternative=alternative) # default is 2 tailed
  
  return norm1, norm2, homo, df, test

def paired_ttest(data1, data2, alternative='two-sided', axis=None):
  norm1 = stats.shapiro(data1.flatten())
  norm2 = stats.shapiro(data2.flatten())

  n = data1.flatten().shape[0]
  df = n - 1 

  test = stats.ttest_rel(a=data1, b=data2, alternative=alternative, axis=axis) # default is 2 tailed
  
  return norm1, norm2, df, test

def getbetweenfc(path, filestr):

  directory = os.fsencode(path)

  for i, folder in enumerate(os.listdir(directory)):
    foldername = os.fsdecode(folder)
    if foldername == 'desktop.ini':
      continue
    subpath = path + '/' + foldername
    subdirec = os.fsencode(subpath)

    if foldername.startswith(filestr) and foldername.endswith(".mat"):
      fcsub = io.loadmat(subpath)

  return fcsub

def findSlope(datax, datay1, datay2, freq=0):
  slopeList = []

  for i in range(10):
      y = np.array([datay1[i, 1, freq], datay2[i, 1, freq]]).flatten().astype(np.float64)
      slope, _, _, _, _ = stats.linregress(datax, y)
      slopeList.append(slope)
  
  return slopeList

def findSlopepfc(datax, datay1, datay2, freq=0):
  slopeList = []

  for i in range(10):
      y = np.array([datay1[0, i, freq], datay2[0, i, freq]]).flatten().astype(np.float64)
      slope, _, _, _, _ = stats.linregress(datax, y)
      slopeList.append(slope)
  
  return slopeList

## PSD Estimation in MNE

In [ ]:
# import simulation result
filesimname = 'NI_72_9.mat'

cutoff = 2000
stim_start = 60000 
stim_end = 120000
total = 180000

# prepare data
filepath = save_path + '/' + filesimname
subs = io.loadmat(filepath)['subs_data'].mean(axis=1) # mean signal from both excitatory and inhibitory subpopulations
subs_time =io.loadmat(filepath)['subs_time'][0]

In [ ]:
# Create metadata
n_channels = 68
sampling_freq = 256  # in Hertz # from 46080 (samples) / 180 sec (simulation length)
info = mne.create_info(n_channels, sfreq=sampling_freq)

all_subs = divide_data (subs, subs_time, total, cutoff=cutoff, stim_start=stim_start, stim_end=stim_end)

raw = mne.io.RawArray(all_subs[0].T, info)
# read raw simulation results (no epoch)
rawpre = mne.io.RawArray(all_subs[1].T, info) # cutoff another 2 sec
rawstim = mne.io.RawArray(all_subs[2][:-512].T, info) # cutoff 4 sec
rawpost = mne.io.RawArray(all_subs[3][:-512].T, info) # cutoff 4 sec

# raw for 20-s epochs
# rawpre = mne.io.RawArray(all_subs[1][4608:].T, info) # cutoff another 18 sec
# rawstim = mne.io.RawArray(all_subs[2][:].T, info)
# rawpost = mne.io.RawArray(all_subs[3][:].T, info)

In [ ]:
# compute PSD on raw data
rawobj_all, rawall_psd = computed_psd_raw(raw)
rawobj_pre, rawpre_psd = computed_psd_raw(rawpre)
rawobj_stim, rawstim_psd = computed_psd_raw(rawstim)
rawobj_post, rawpost_psd = computed_psd_raw(rawpost)

# compute PSD for each epoch
# eduration = 20

# epochs = mne.make_fixed_length_epochs(raw, duration=eduration, preload=True)
# prestim = mne.make_fixed_length_epochs(rawpre, duration=eduration, preload=False)
# stimon = mne.make_fixed_length_epochs(rawstim, duration=eduration, preload=False)
# poststim = mne.make_fixed_length_epochs(rawpost, duration=eduration, preload=False)

# epoobj_all, epoall_psd = computed_psd_eachEpochs(epochs)
# epoobj_pre, prestim_psd = computed_psd_eachEpochs(prestim)
# epoobj_stim, stimon_psd = computed_psd_eachEpochs(stimon)
# epoobj_post, poststim_psd = computed_psd_eachEpochs(poststim)

In [ ]:
# export raw PSD
psdrawresult = save_path + '/PSDrawpfcResult_' + subject + '_' + id + '.mat'
io.savemat(psdrawresult, mdict={
  'psd raw all': np.array((rawall_psd), dtype=object),
  'psd raw prestim': np.array((rawpre_psd), dtype=object),
  'psd raw stimon': np.array((rawstim_psd), dtype=object),
  'psd raw poststim': np.array((rawpost_psd), dtype=object)
})

# export epochs PSD
# psdepresult = save_path + '/PSDepochspfc20sResult_' + subject + '_' + id + '.mat'
# io.savemat(psdepresult, mdict={
#   'psd epo all': np.array((epoall_psd), dtype=object),
#   'psd epo prestim': np.array((prestim_psd), dtype=object),
#   'psd epo stimon': np.array((stimon_psd), dtype=object),
#   'psd epo poststim': np.array((poststim_psd), dtype=object)
# })

## Compute Relative power

In [ ]:
# import PSD
psdfilename = ''
psdpath = save_path + '/' + psdfilename
sub_psd = io.loadmat(psdpath)

prefix_rawpsd = ['psd raw all', 'psd raw prestim', 'psd raw stimon', 'psd raw poststim']

In [ ]:
# from all nodes
eegpowraw_all, pathtopowerraw = powerEEGraw(sub_psd, prefix=prefix_rawpsd, mode='allNodes')

In [ ]:
# import all subjects
youngpower = getallEEGpow(path=y_folder, filestr=filestr)
oldpower = getallEEGpow(path=o_folder, filestr=filestr)

# mean relative power of 10 rounds for each node
prefix_pow = ['power all', 'power pre', 'power stim', 'power post']

### Test relative power

In [ ]:
# mean power across all nodes for each round
mean_youngpower = mean_power(data=youngpower, prefix=prefix_pow, axis=1)
mean_oldpower = mean_power(data=oldpower, prefix=prefix_pow, axis=1)

testdata1 = mean_youngpower.copy()
testdata2 = mean_oldpower.copy()

# test normality
swapow_pre_normy = kstest(data=testdata1[prefix_pow[1]][:, 1, 0].flatten().astype(np.float64))
ospow_pre_normy = kstest(data=testdata1[prefix_pow[1]][:, 1, 1].flatten().astype(np.float64))
delpow_pre_normy = kstest(data=testdata1[prefix_pow[1]][:, 1, 2].flatten().astype(np.float64))
swapow_pre_normo = kstest(data=testdata2[prefix_pow[1]][:, 1, 0].flatten().astype(np.float64))
ospow_pre_normo = kstest(data=testdata2[prefix_pow[1]][:, 1, 1].flatten().astype(np.float64))
delpow_pre_normo = kstest(data=testdata2[prefix_pow[1]][:, 1, 2].flatten().astype(np.float64))

# calculate median and interquartile range (IQR)
data1 = np.array([testdata1[p][:, 1, :3] for p in prefix_pow[1:]]).astype(np.float64) # data1 = (session, rounds, frequency)
data2 = np.array([testdata2[p][:, 1, :3] for p in prefix_pow[1:]]).astype(np.float64)
med = np.median(data1[2, :, 0])
q1, q3 = np.percentile(data1[2, :, 0], [25, 75])
iqr = q3 - q1

# Mann-Whitney U test
# t-test young has SWA more than old --> young (data1) greater than old (data2)
testpremyoswa = stats.mannwhitneyu(testdata1[prefix_pow[1]][:, 1, 0].astype(np.float64), testdata2[prefix_pow[1]][:, 1, 0].astype(np.float64), axis=0, alternative='greater')
testpremyoos = stats.mannwhitneyu(testdata1[prefix_pow[1]][:, 1, 1].astype(np.float64), testdata2[prefix_pow[1]][:, 1, 1].astype(np.float64), axis=0, alternative='greater')
testpremyodel = stats.mannwhitneyu(testdata1[prefix_pow[1]][:, 1, 2].astype(np.float64), testdata2[prefix_pow[1]][:, 1, 2].astype(np.float64), axis=0, alternative='greater')

In [ ]:
# mean power across PFC nodes for each round
mstdpfc_ypow = meanstdpfc_power(data=youngpower, prefix=prefix_pow)
mstdpfc_opow = meanstdpfc_power(data=oldpower, prefix=prefix_pow)

# test normality
testdata1 = mstdpfc_ypow.copy()
testdata2 = mstdpfc_opow.copy()

swapow_pre_normy = kstest(data=testdata1[prefix_pow[1]][0, :, 0].flatten().astype(np.float64))
ospow_pre_normy = kstest(data=testdata1[prefix_pow[1]][0, :, 1].flatten().astype(np.float64))
delpow_pre_normy = kstest(data=testdata1[prefix_pow[1]][0, :, 2].flatten().astype(np.float64))
swapow_pre_normo = kstest(data=testdata2[prefix_pow[1]][0, :, 0].flatten().astype(np.float64))
ospow_pre_normo = kstest(data=testdata2[prefix_pow[1]][0, :, 1].flatten().astype(np.float64))
delpow_pre_normo = kstest(data=testdata2[prefix_pow[1]][0, :, 2].flatten().astype(np.float64))

# calculate median and interquartile range (IQR)
data1 = np.array([testdata1[p][0, :, :3] for p in prefix_pow[1:]]).astype(np.float64) 
data2 = np.array([testdata2[p][0, :, :3] for p in prefix_pow[1:]]).astype(np.float64)
med = np.median(data1[2, :, 0])
q1, q3 = np.percentile(data1[2, :, 0], [25, 75])
iqr = q3 - q1

# Mann-Whitney U test
# t-test young has SWA more than old --> young (data1) greater than old (data2)
testprempfcyoswa = stats.mannwhitneyu(testdata1[prefix_pow[1]][0, :, 0].astype(np.float64), testdata2[prefix_pow[1]][0, :, 0].astype(np.float64), axis=0, alternative='greater')
testprempfcyoos = stats.mannwhitneyu(testdata1[prefix_pow[1]][0, :, 1].astype(np.float64), testdata2[prefix_pow[1]][0, :, 1].astype(np.float64), axis=0, alternative='greater')
testprempfcyodel = stats.mannwhitneyu(testdata1[prefix_pow[1]][0, :, 2].astype(np.float64), testdata2[prefix_pow[1]][0, :, 2].astype(np.float64), axis=0, alternative='greater')

## Effect of so-tDCS over time

In [ ]:
# import PSD
psdfilename = ''
psdpath = save_path + '/' + psdfilename
sub_psd = io.loadmat(psdpath)

# compute PSD for each epoch
prefix_epopsd = ['psd epo all', 'psd epo prestim', 'psd epo stimon', 'psd epo poststim']
eegpowepo_all, pathtopowerepo = powerEEGepo(data=sub_psd, prefix=prefix_epopsd)

In [ ]:
# import all subjects
youngpower = getallEEGpow(path=y_folder, filestr=filestr)
oldpower = getallEEGpow(path=o_folder, filestr=filestr)

# combine young & old into 1 array before computing z-score from 20 rounds
# all nodes
allpower = {}
prefix_pow = ['power all', 'power pre', 'power stim', 'power post']
for i in prefix_pow:
  allpower[i] = np.concatenate((youngpower[i][:, :, :, 0, 0], oldpower[i][:, :, :, 0, 0]), axis=0)

# Convert all absolute power into z-score
zwhole_power = convert_zscore(data=allpower, prefix=prefix_pow, axis=None)

In [ ]:
# test normality with Kolmogorov-Smirnov (KS) test
# z-score from whole array
testdata = zwhole_power

swa_norm = {}
normo = []
normy = []
for i in range(8):
  swazwh_normy = kstest(data=testdata[:10, i, :].flatten().astype(np.float64))
  swazwh_normo = kstest(data=testdata[10:, i, :].flatten().astype(np.float64))
  normy.append(swazwh_normy)
  normo.append(swazwh_normo)

swa_norm['young'] = normy
swa_norm['old'] = normo

# median (interquartile range [IQR])
y = testdata[:10, :, :]
o = testdata[10:, :, :]
med_y = np.median(y, axis=0)
med_o = np.median(o, axis=0)

q13y = np.percentile(y, [25, 75], axis=0)
q13o = np.percentile(o, [25, 75], axis=0)
iqry = q13y[1] - q13y[0]
iqro = q13o[1] - q13o[0]

# test SWA all nodes with two-tailed Mann-Whitney U test 
# pre
testyozswa = stats.mannwhitneyu(testdata[:10, :, :], testdata[10:, :, :])

### Test slope of relative power
- slope pre-stim and stim-on
- slope stim-on and post-stim

In [ ]:
# global SWA
yps_slopeswa = findSlope(datax=[0,1], datay1=mean_youngpower[prefix_pow[1]], datay2=mean_youngpower[prefix_pow[2]])
ysp_slopeswa = findSlope(datax=[1,2], datay1=mean_youngpower[prefix_pow[2]], datay2=mean_youngpower[prefix_pow[3]])
ops_slopeswa = findSlope(datax=[0,1], datay1=mean_oldpower[prefix_pow[1]], datay2=mean_oldpower[prefix_pow[2]])
osp_slopeswa = findSlope(datax=[1,2], datay1=mean_oldpower[prefix_pow[2]], datay2=mean_oldpower[prefix_pow[3]])

dataslope = np.array([yps_slopeswa, ysp_slopeswa, ops_slopeswa, osp_slopeswa])

# ks-test
swapow_ps_normy = kstest(data=dataslope[0])
swapow_sp_normy = kstest(data=dataslope[1])
swapow_ps_normo = kstest(data=dataslope[2])
swapow_sp_normo = kstest(data=dataslope[4])

# Mann-whiteney test
mtestpsyoswa = stats.mannwhitneyu(dataslope[0], dataslope[2], axis=0)
mtestspyoswa = stats.mannwhitneyu(dataslope[1], dataslope[3], axis=0)

In [ ]:
# PFC SWA
yps_slopeswa = findSlopepfc(datax=[0,1], datay1=mstdpfc_ypow[prefix_pow[1]], datay2=mstdpfc_ypow[prefix_pow[2]])
ysp_slopeswa = findSlopepfc(datax=[1,2], datay1=mstdpfc_ypow[prefix_pow[2]], datay2=mstdpfc_ypow[prefix_pow[3]])
ops_slopeswa = findSlopepfc(datax=[0,1], datay1=mstdpfc_opow[prefix_pow[1]], datay2=mstdpfc_opow[prefix_pow[2]])
osp_slopeswa = findSlopepfc(datax=[1,2], datay1=mstdpfc_opow[prefix_pow[2]], datay2=mstdpfc_opow[prefix_pow[3]])

dataslope = np.array([yps_slopeswa, ysp_slopeswa, ops_slopeswa, osp_slopeswa])

# ks-test
swapfc_ps_normy = kstest(data=dataslope[0])
swapfc_sp_normy = kstest(data=dataslope[1])
swapfc_ps_normo = kstest(data=dataslope[2])
swapfc_sp_normo = kstest(data=dataslope[4])

# Mann-whiteney test
mtestpspfcyoswa = stats.mannwhitneyu(dataslope[0], dataslope[2], axis=0)
mtestsppfcyoswa = stats.mannwhitneyu(dataslope[1], dataslope[3], axis=0)

## Functional Connectivity

In [ ]:
# load a simulated BOLD
bold_data = io.loadmat(filepath)['bold_data']
bold_time = io.loadmat(filepath)['bold_time'][0]

# segment data 
all_bolds = divide_data(subs=bold_data, subs_time=bold_time, total=total, cutoff=cutoff, stim_start=stim_start, stim_end=stim_end)
prefix = ['all data', 'pre-stim', 'stim-on', 'post-stim']

# compute simulated FC
# use method without remove upper triangle because they didn't done that in reference work
simbold = np.squeeze(all_bolds[0][:, :, :, :].mean(axis=(1,3))).T
all_simFC = np.corrcoef(simbold)
all_simFC[np.diag_indices_from(all_simFC)] = 0

In [ ]:
# computed fc matrix for all, pre, stim post from all nodes & pfc

data = [all_bolds[0][:, :, chosen_PFC, :], all_bolds[1][:, :, chosen_PFC, :], all_bolds[2][:, :, chosen_PFC, :], all_bolds[3][:, :, chosen_PFC, :]]
prefix_fc = ['pfc fc', 'pfc pre fc', 'pfc stim fc', 'pfc post fc']
all_fcs = {}

for i in range(8):
  bold = np.squeeze(data[i].mean(axis=(1,3))).T
  all_simFC = np.corrcoef(bold)
  all_simFC[np.diag_indices_from(all_simFC)] = 0

  all_fcs[prefix_fc[i]] = all_simFC

In [ ]:
# correlation pre-stim, pre-post, stim-post fc for each round --> save correlation of each round
allfc = getallfc(path=save_path, filestr=filestr)

prefc_tri = prep_FC(allfc[prefix_fc[1]])
stimfc_tri = prep_FC(allfc[prefix_fc[2]])
postfc_tri = prep_FC(allfc[prefix_fc[3]])

corr_all = {}
ps = []
pp = []
sp = []

for i in range(10):
  corfc_prestim = np.corrcoef(prefc_tri[i].astype(np.float64), stimfc_tri [i].astype(np.float64))[0,1]
  corfc_prepost = np.corrcoef(prefc_tri[i].astype(np.float64), postfc_tri[i].astype(np.float64))[0,1]
  corfc_stimpost = np.corrcoef(stimfc_tri[i].astype(np.float64), postfc_tri[i].astype(np.float64))[0,1]
  ps.append(corfc_prestim)
  pp.append(corfc_prepost)
  sp.append(corfc_stimpost)

corr_all['cor pre-stim'] = ps
corr_all['cor pre-post'] = pp
corr_all['corr stim-post'] = sp 

# compute mean correlation on PFC FC
mean_corr = np.vstack([np.array(prefc_tri).mean(axis=1), np.array(stimfc_tri).mean(axis=1), np.array(postfc_tri).mean(axis=1)])

In [ ]:
# t-test on correlation of correlation
youngfc = getbetweenfc(path=y_folder, filestr=filestr)
oldfc = getbetweenfc(path=o_folder, filestr=filestr)

prefix = ['correlation pre-stim PFC', 'correlation pre-post PFC', 'correlation stim-post PFC', 'mean correlation PFC FC 10 rounds']

_, _, homopspfc, dfpspfc, testpspfc = ind_ttest(youngfc[prefix[0]], oldfc[prefix[0]], axis=1)
_, _, homopppfc, dfpppfc, testpppfc = ind_ttest(youngfc[prefix[1]], oldfc[prefix[1]], axis=1)
_, _, homosppfc, dfsppfc, testsppfc = ind_ttest(youngfc[prefix[2]], oldfc[prefix[2]], axis=1)

# t-test on mean correlation between subject
_, _, homoprepfc, dfprepfc, testprepfc = ind_ttest(youngfc[prefix[3]][0, :].flatten().astype(np.float64), oldfc[prefix[3]][0, :].flatten().astype(np.float64), axis=0)
_, _, homostimpfc, dfstimpfc, teststimpfc = ind_ttest(youngfc[prefix[3]][1, :].flatten().astype(np.float64), oldfc[prefix[3]][1, :].flatten().astype(np.float64), axis=0)
_, _, homopostpfc, dfpostpfc, testpostpfc = ind_ttest(youngfc[prefix[3]][2, :].flatten().astype(np.float64), oldfc[prefix[3]][2, :].flatten().astype(np.float64), axis=0)

# t-test on mean correlation within subject
# young
a, b, dfprepfc, testprepfc = paired_ttest(youngfc[prefix[3]][0, :].flatten().astype(np.float64), youngfc[prefix[3]][1, :].flatten().astype(np.float64))
c, d, dfstimpfc, teststimpfc = paired_ttest(youngfc[prefix[3]][1, :].flatten().astype(np.float64), youngfc[prefix[3]][2, :].flatten().astype(np.float64))
e, f, dfpostpfc, testpostpfc = paired_ttest(youngfc[prefix[3]][0, :].flatten().astype(np.float64), youngfc[prefix[3]][2, :].flatten().astype(np.float64))
# old
a, b, dfprepfc, testprepfc = paired_ttest(oldfc[prefix[3]][0, :].flatten().astype(np.float64), oldfc[prefix[3]][1, :].flatten().astype(np.float64))
c, d, dfstimpfc, teststimpfc = paired_ttest(oldfc[prefix[3]][1, :].flatten().astype(np.float64), oldfc[prefix[3]][2, :].flatten().astype(np.float64))
e, f, dfpostpfc, testpostpfc = paired_ttest(oldfc[prefix[3]][0, :].flatten().astype(np.float64), oldfc[prefix[3]][2, :].flatten().astype(np.float64))